In [2]:
import cv2
import numpy as np
import imutils
import glob
import matplotlib.pyplot as plt
import os
import time

In [3]:
def Kalman(vars,dim):

    PredVec=vars*dim

    kalman = cv2.KalmanFilter(PredVec, dim) #6,3 Which states for position and velocity in 3D (x,y,z,vx,vy,vz)

    kalman.measurementMatrix = np.array([[1,0,0,0,0,0],             #Z
                                         [0,0,1,0,0,0],
                                         [0,0,0,0,1,0]], np.float32)


    kalman.transitionMatrix = np.array([[1,1,0,0,0,0],              #F
                                        [0,1,0,0,0,0],
                                        [0,0,1,1,0,0],
                                        [0,0,0,1,0,0],
                                        [0,0,0,0,1,1],
                                        [0,0,0,0,0,1]], np.float32)
                                        
    kalman.processNoiseCov = np.eye(6, dtype=np.float32) * 0.05
    kalman.measurementNoiseCov = np.eye(3, dtype=np.float32) * 0.12

    prediction = np.zeros((PredVec,1), np.float32)

    return prediction, kalman

In [4]:
def BkgRmv(sel=1):
    if sel == 0:
        print("BackGroundSubstractor KNN")
        fgbg = cv2.createBackgroundSubtractorKNN(history=1000, dist2Threshold=1000, detectShadows=False)
    elif sel== 1:
        print("BackGroundSubstractor Gaussian mixture model")
        fgbg = cv2.createBackgroundSubtractorMOG2()
    elif sel == 2:
        print(" BackGroundSubstractor Geometric Multigrid")
        print(" NEEDS TO BEINSTALLED SEPARATEDLY")
        fgbg = cv2.bgsegm.createBackgroundSubtractorGMG()
    else: 
        print("Frame Difference method")
        """"
        if frameNum == 1:
    	bgFrame = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
        elif frameNum > 1:
            foreFrame = cv2.cvtColor(tmp, cv2.COLOR_BGR2GRAY)
            foreFrame = cv2.absdiff(foreFrame, bgFrame)
            _, thresh = cv2.threshold(foreFrame, 30, 255, cv2.THRESH_BINARY)
            gaussian = cv2.GaussianBlur(thresh, (3, 3), 0)
            cv2.imshow('gaussian', foreFrame)
        """""
    return fgbg

In [5]:
def sum_abs_diff(image1, image2):
    image1 = image1.astype('int32')
    image2 = image2.astype('int32')
    
    sad = 0 #Sum of Absolute Differences
    
    if image1.shape == image2.shape:
        diff = image1 - image2
        sad = np.sum(np.absolute(diff))
    else:
        sad = -1
    
    return sad

def scan_line(span, template, search_col_min, search_col_max):
    min_place = -1
    min_value = float('inf')
    for i in range(search_col_min, search_col_max):
        diff = sum_abs_diff(span[:, i:i + span.shape[0]], template)
        if diff < min_value:
            min_value = diff
            min_place = i
    return (min_place, min_value)

def Depth(leftGrayImg, rightGrayImg, positionOnLeft, halfWindow):
    baseline = 120
    focalLength = 700
    
    template = leftGrayImg[positionOnLeft[1]-halfWindow:positionOnLeft[1]+halfWindow,\
                           positionOnLeft[0]-halfWindow:positionOnLeft[0]+halfWindow]    
    span = rightGrayImg[positionOnLeft[1]-halfWindow:positionOnLeft[1]+halfWindow, :]
    min_place, min_value = scan_line(span, template, positionOnLeft[0]-250, positionOnLeft[0]-50-halfWindow)
    
    if min_place < 0:
        print('fail')
        
    disparity = positionOnLeft[0]-min_place
    #print(f"Disparity: {disparity} pixels")
    
    depth = focalLength*baseline/disparity
    #print(f"Depth of pixel [{positionOnLeft[0]},{positionOnLeft[1]}] in mm: {depth}")
    return depth

In [6]:
path_to_calib_params = "calibration_params/"

mtx_l = np.loadtxt(os.path.join(path_to_calib_params, 'mtx_l.csv'), delimiter=',')
dist_l = np.loadtxt(os.path.join(path_to_calib_params, 'dist_l.csv'), delimiter=',')
mtx_r = np.loadtxt(os.path.join(path_to_calib_params, 'mtx_r.csv'), delimiter=',')
dist_r = np.loadtxt(os.path.join(path_to_calib_params, 'dist_r.csv'), delimiter=',')
R = np.loadtxt(os.path.join(path_to_calib_params, 'R.csv'), delimiter=',')
T = np.loadtxt(os.path.join(path_to_calib_params, 'T.csv'), delimiter=',')

In [7]:
def rectify_image_pair(img_l, img_r, mtx_l, dist_l, mtx_r, dist_r, R, T):
    size = img_l.shape[:2]
    size = size[::-1]
    
    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(mtx_l, dist_l, mtx_r, dist_r, size, R, T)
    
    mapl1, mapl2 = cv2.initUndistortRectifyMap(mtx_l, dist_l, R1, P1, (img_l.shape[:2])[::-1], cv2.CV_32FC1)
    mapr1, mapr2 = cv2.initUndistortRectifyMap(mtx_r, dist_r, R2, P2,(img_r.shape[:2])[::-1], cv2.CV_32FC1)
    
    iml = cv2.remap(img_l, mapl1, mapl2, cv2.INTER_LINEAR)
    imr = cv2.remap(img_r, mapr1, mapr2, cv2.INTER_LINEAR)
    
    return iml, imr

In [12]:
rightSet = sorted(glob.glob('./../../Data/Stereo_conveyor_with_occlusions/right/*.png')) #= cv2.VideoCapture(CamL_id)

#Create Kalman Filter for state prediction
prediction, kalman = Kalman(2,3)
 
#Create Background Removal
fgbg = BkgRmv(sel=0)

#Preprocess image?
morph = True
crop = True

# New Object?
thresholdCenter = 10
prevCenter = (0, 0)
prevRadius = 0
thresholdRadius = 5

# Crop the image?
cropTop = 220
cropBottom = 660
cropLeft = 180
cropRight = 1140

for rightName in rightSet:
   printed = False
   print("Next Frame")
   leftName = rightName.replace('right','left').replace('Right','Left')
   imgright = cv2.imread(rightName)
   imgleft = cv2.imread(leftName)
   imrght = cv2.cvtColor(imgright,cv2.COLOR_BGR2GRAY)
   imglft = cv2.cvtColor(imgleft, cv2.COLOR_BGR2GRAY)
   
   #Attempt with rectified. TOO MUCH FOV LOSS
   img_l_rect, img_r_rect = rectify_image_pair(imgleft, imgright, mtx_l, dist_l, mtx_r, dist_r, R, T)
   imrght = cv2.cvtColor(imgright,cv2.COLOR_BGR2GRAY)
   imglft = cv2.cvtColor(imgleft, cv2.COLOR_BGR2GRAY)

   #Attempt with rectified. A BIT BETTER, NOT ENOUGH
   #img_l_rect = cv2.undistort(imgleft, mtx_l, dist_l, None, None)
   #img_r_rect = cv2.undistort(imgright, mtx_r, dist_r, None, None)
   imrght = cv2.cvtColor(imgright,cv2.COLOR_BGR2GRAY)
   imglft = cv2.cvtColor(imgleft, cv2.COLOR_BGR2GRAY)


   originalFrame = imgright.copy()
   originalFrame2 = imgright.copy()

   
   if morph == True:
      # Smoothening of the image 
      # Morphological Operations: Erosion +  Dilation
      kernel = np.ones((3,3), np.uint8)
      kernel = np.ones((5,5), np.uint8) 
      imrght = cv2.morphologyEx(imrght,cv2.MORPH_OPEN,kernel)
      imrght = cv2.morphologyEx(imrght,cv2.MORPH_OPEN,kernel)  #Open = Erosion + Dilation
   
   filtered = fgbg.apply(imrght)
   filtered = cv2.morphologyEx(filtered,cv2.MORPH_OPEN,kernel)
   filtered = cv2.morphologyEx(filtered,cv2.MORPH_DILATE,kernel)

   if crop == True:
      #Focus the analysis in the area of interest
      cropped = filtered[cropTop:cropBottom, cropLeft:cropRight]

   cnts, _ = cv2.findContours(cropped.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
   #contours =np.zeros(filtered.shape)
   #contours=cv2.drawContours(contours, cnts,-1, 255, 1)

   if len(cnts) > 0:
      # In case contours were found..
      # Find the largest one and return centroid
      c = max(cnts, key=cv2.contourArea)
      ((x, y), radius) = cv2.minEnclosingCircle(c)
      M = cv2.moments(c) #Find the center of the shape
      if  not M["m00"] == 0.0:
                #If the center of the biggest object in the filtered image is in the origin stop
      #Recovering of the global positions
         cX = int(M["m10"] / M["m00"]) + cropLeft 
         cY = int(M["m01"] / M["m00"]) + cropTop
         center = (cX,cY)

         if radius > 40 and radius < 200 and radius + thresholdRadius >= prevRadius:
         # If the object meets the size requirements...
            cv2.circle(originalFrame, (int(cX), int(cY)), int(radius),
                     (0, 255, 255), 2)                             # Enclsoing circle
            cv2.circle(originalFrame, center, 5, (0, 0, 255), -1)   # Center of mass
            
            #In order to compute the depth, we will use the stereo pair
            #Then this depth will be used by the kalman filter for 3D state estimation
            print("Computing depth")
            
            depth = Depth(imglft, imrght, center, 50) # We pass the center of gravity of the object to the estimate it's depth
            print(depth)
            
            cv2.putText(originalFrame, f"Depth: {depth:.2f} mm", (0,200), cv2.FONT_ITALIC, 1, (0,0,0))
            
            prevRadius = radius
            text = "Object found at:"
            colour = (0, 255, 0)
            cv2.putText(originalFrame,"({}, {})".format(int(center[0]), int(center[1])), (0, 100), cv2.FONT_ITALIC, 1, colour)
            printed = True
            if (center[0] > prevCenter[0] + thresholdCenter) and (center[1] < prevCenter[1] - thresholdCenter):
               prediction, kalman = Kalman(2,3) #Restart the kalman filter every time a new object is found
               prevRadius = 0
            #Once that the object has been found we start the correction phase of the Kalman filter
            kalman.correct(np.array([np.float32(center[0]), np.float32(center[1]), np.float32(depth)], np.float32))
            prevCenter = center
         else: 
            # Object not found
            text = "Object not found"
            objectFound = False
            colour = (0, 0, 255)
            prevRadius = 0
            printed = True
            cv2.putText(originalFrame, text, (0, 50), cv2.FONT_ITALIC, 1, colour)
      else:
         cv2.putText(originalFrame, "PREDICTION", (0, 50), cv2.FONT_ITALIC, 1, colour)
         prediction = kalman.predict()
         center_pred = (int(prediction[0]+prediction[1]),int(prediction[2]+prediction[3]))
         
         cv2.circle(originalFrame, center_pred, 5, (0, 255, 0), -1)
         cv2.putText(originalFrame,"({}, {}, {})".format(prediction[0], prediction[2], prediction[4]), 
                     (0, 150), cv2.FONT_ITALIC, 1, colour)
         printed = True
      if not printed:
      # Object not found
         text = "Object not found"
         objectFound = False
         colour = (0, 0, 255)
         prevRadius = 0   
         cv2.putText(originalFrame, text, (0, 50), cv2.FONT_ITALIC, 1, colour)
   #else:
   #We will only make predictions when the object is not found
      
      
   #group = np.concatenate((originalFrame, filtered[...,np.newaxis]),axis=1)
   #cv2.imshow('Video', filtered)
   cv2.imshow('Video', originalFrame)
   if cv2.waitKey(30) & 0xFF == ord('q'):
      cv2.destroyAllWindows()
      cv2.waitKey(1)
      break
         

BackGroundSubstractor KNN
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Next Frame
Computing depth
831.6831683168317
Next Frame
Computing depth


In [ ]:
print(filtered.shape)

(720, 1280)


In [ ]:
while True:
    GRID_SIZE = 20
    height, width, channels = originalFrame.shape
    for x in range(0, width -1, GRID_SIZE):
        cv2.line(originalFrame, (x, 0), (x, height), (255, 0, 0), 1, 1)
    for y in range(0,height-1,GRID_SIZE):
        cv2.line(originalFrame, (0, y), (width, y), (0, 255, 0), 1, 1)
    cv2.imshow('res',originalFrame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        
        break